In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from typing import Tuple, Optional, Dict
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
import wandb

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from research.data.natural_scenes import (
    NaturalScenesDataset,
    StimulusDataset,
    KeyDataset
)
from research.models.components_2d import BlurConvTranspose2d
from research.models.fmri_decoders import VariationalDecoder, SpatialDecoder, SpatialDiscriminator, Decoder
from research.models.fmri_encoders import Encoder, SpatialEncoder
from research.metrics.loss_functions import (
    EuclideanLoss,
    EmbeddingClassifierLoss,
    ProbabalisticCrossEntropyLoss,
    VariationalLoss,
    CosineSimilarityLoss,
    EmbeddingDistributionLoss,
    ContrastiveDistanceLoss,
)
from research.experiments.nsd_experiment import NSDExperiment
from research.metrics.metrics import (
    cosine_similarity, 
    r2_score,
    pearsonr,
    embedding_distance,
    cosine_distance,
    squared_euclidean_distance,
    contrastive_score,
    two_versus_two,
    smooth_euclidean_distance,
)
from pipeline.utils import product

In [2]:
nsd_path = Path('D:\\Datasets\\NSD\\')
nsd = NaturalScenesDataset(nsd_path)

In [3]:
def run_experiment(
        train_dataset: Dataset,
        val_dataset: Dataset,
        batch_size: int,
        channels_last: bool,
        group: str = None,
        max_iterations: int = 10001,
        evaluation_interval: int = 250,
        notes: str = None,
        config: Dict = None,
        wandb_logging: bool = False,
):
    if config is None:
        config = {}
    device = torch.device('cuda')
    
    sample = train_dataset[0]
    betas_shape = sample['betas'][0].shape
    stimulus_shape = sample['stimulus']['data'].shape
    print(f'{betas_shape=}, {stimulus_shape=}')
    
    if len(stimulus_shape) == 1:
        model_params = dict(
            layer_sizes=[
                product(stimulus_shape),
                betas_shape[0]
            ],
        )
        model = Encoder(**model_params)
        model.to(device)
    elif len(stimulus_shape) == 3:
        model_params = dict(
            input_shape=stimulus_shape,
            output_size=betas_shape[0],
            channels_last=channels_last,
        )
        model = SpatialEncoder(**model_params)
        model.to(device)
    
    criterion_params = dict()
    criterion = nn.MSELoss(**criterion_params)
    
    optimizer_params = dict(lr=1e-3)
    optimizer = Adam(
        params=model.parameters(),
        **optimizer_params,
    )
    
    training_params = dict(
        batch_size=batch_size,
        evaluation_interval=evaluation_interval,
        evaluation_subset_size=500,
    )
    experiment = NSDExperiment(
        mode='encode',
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        device=device,
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        **training_params
    )

    config = {
        **config,
        'model': model,
        **model_params,
        'criterion': criterion,
        **criterion_params,
        'optimizer': optimizer,
        **optimizer_params,
        **training_params,
    }
    wandb.init(project='nsd-encoding', config=config, group=group, notes=notes)
    wandb.define_metric("*", summary="max")
    wandb.define_metric("*", summary="min")

    experiment.train_model(max_iterations=max_iterations, logger=wandb.log)
    return experiment


In [4]:
wandb_logging = False,

run_models = [
    #('ViT-B=32', 'embedding'),
    ('ViT-B=32-text', 'embedding_mean'),
    #('ViT-B=32', 'transformer.resblocks.3'),
    #('bigbigan-resnet50', 'z_mean'),
    #('DPT_Large', 'scratch.refinenet4'),
]

subjects = nsd.subjects.keys()
#subjects = [f'subj0{i}' for i in range(1, 9)]

for model_name, stimulus_key in run_models:
    for subject_name in subjects:
        num_folds = 5
        notes = None

        experiment_params = dict(
            batch_size=128,
            group='group-1',
            max_iterations = 2501,
            evaluation_interval = 250,
            channels_last=False#(model_name == 'ViT-B=32' and stimulus_key != 'embedding'),
        )

        betas_params = dict(
            subject_name=subject_name,
            voxel_selection_path='derivatives/voxel-selection.hdf5',
            voxel_selection_key='nc/value',
            threshold=5.,
            return_volume_indices=True
        )
        betas, betas_indices = nsd.load_betas(**betas_params)

        stimulus_params = dict(
            subject_name=subject_name,
            #stimulus_path='nsddata_stimuli/stimuli/nsd/nsd_stimuli.hdf5',
            #stimulus_key='imgBrick',
            stimulus_path=f'derivatives/stimulus_embeddings/{model_name}.hdf5',
            stimulus_key=stimulus_key,
            delay_loading=True
        )
        stimulus = nsd.load_stimulus(**stimulus_params)

        dataset = KeyDataset({'betas': betas, 'stimulus': stimulus})
        train_dataset, val_dataset, test_dataset = nsd.apply_subject_split(dataset, subject_name, 'split-01')

        Y_cv = []
        Y_pred_cv = []
        for fold_id in range(num_folds):
            fold_params = dict(num_folds=num_folds, select_fold=fold_id)
            cv_train_dataset, cv_val_dataset = nsd.apply_nfold_split(dataset, **fold_params)

            config = {
                'model_name': model_name,
                **betas_params, 
                **stimulus_params, 
                **fold_params
            }

            experiment = run_experiment(
                cv_train_dataset,
                cv_val_dataset,
                **experiment_params,
                config=config
            )

            with torch.no_grad():
                Y, Y_pred, _ = experiment.run_all(experiment.val_dataset)
            Y_cv.append(Y)
            Y_pred_cv.append(Y_pred)

        Y_cv = nsd.combine_nfold_tensors(Y_cv, num_folds=num_folds)
        Y_pred_cv = nsd.combine_nfold_tensors(Y_pred_cv, num_folds=num_folds)
        r2_cv = r2_score(Y_cv, Y_pred_cv, reduction=None, cast_dtype=None)

        config = {'model_name': model_name, **betas_params, **stimulus_params}
        experiment = run_experiment(
            train_dataset,
            val_dataset,
            config=config,
            **experiment_params,
        )

        def require_dataset(group, key, tensor):
            if key in group:
                group[key][:] = tensor
            else:
                group[key] = tensor
        encoded_betas_path = nsd_path / 'derivatives/encoded_betas'
        key_name = wandb.run.group if wandb.run.group else wandb.run.name
        save_file_path = encoded_betas_path / wandb.config['model_name'] / f'{key_name}.hdf5'
        save_file_path.parent.mkdir(exist_ok=True, parents=True)

        h5_key = (wandb.config['subject_name'], wandb.config['stimulus_key'])

        attributes = dict(wandb.config)
        attributes['wandb_run_name'] = wandb.run.name
        attributes['wandb_run_url'] = wandb.run.url
        attributes['wandb_group'] = wandb.run.group
        attributes['wandb_notes'] = wandb.run.notes

        with h5py.File(save_file_path, 'a') as f:
            key = '/'.join(h5_key)
            group = f.require_group(key)
            for k, v in attributes.items():
                group.attrs[k] = v
            group.attrs['iteration'] = experiment.iteration
            require_dataset(group, 'volume_indices', betas_indices)
            require_dataset(group, 'r2', r2_cv)
            require_dataset(group, 'betas_pred', Y_pred_cv)

            model_group = group.require_group('model')
            for param_name, weights in experiment.model.state_dict().items():
                weights = weights.cpu()
                require_dataset(model_group, param_name, weights)

            volume_r2 = nsd.reconstruct_volume(subject_name, r2_cv, betas_indices)

            images_path = nsd_path / 'derivatives' / 'images' / subject_name / 'func1pt8mm' / 'pytorch'
            images_path.mkdir(exist_ok=True, parents=True)

            image_key = (
                subject_name, 'pytorch', key_name, model_name, 
                *wandb.config['stimulus_key'].split('/'),
                'r2',
            )
            image_file_name = '__'.join(image_key) + '.nii.gz'
            image_path = images_path / image_file_name

            affine = nsd.get_affine(subject_name)
            image = nib.Nifti1Image(volume_r2.T.numpy(), affine)
            nib.save(image, image_path)

betas_shape=torch.Size([27790]), stimulus_shape=torch.Size([512])


wandb: Currently logged in as: efirdc (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2501/2501 [01:35<00:00, 26.08it/s]


betas_shape=torch.Size([27790]), stimulus_shape=torch.Size([512])


loss,▅█▆▂▄▅▄▇▅▅▂▃▅▄▆▁█▄▄▂▁▅▄▄▆▂▄▃▄▃▂▆▂▃▂▂▃▃▅▄


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2501/2501 [01:04<00:00, 38.85it/s]


betas_shape=torch.Size([10090]), stimulus_shape=torch.Size([512])


loss,


wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2501/2501 [01:03<00:00, 39.19it/s]


In [ ]:
r2_cv = r2_score(Y_cv[0], Y_pred_cv[0], reduction=None, cast_dtype=None)
r2_cv.max()

In [4]:
nsd_path

WindowsPath('D:/Datasets/NSD')

In [3]:
subject_name = 'subj01'
fracridge_r2_ = f'{subject_name}__fracridge__ViT-B=32__embedding__r2.nii.gz'
images_path = nsd_path / f'derivatives/images/{subject_name}/func1pt8mm'
fracridge_r2 = fracridge'
fracridge_r2 = nib.load(fracridge_r2).get_fdata()

diff = fracridge_r2 - volume_r2.T.numpy()
@interact(d=(0, diff.shape[2]-1))
def show(d):
    plt.figure(figsize=(10, 10))
    plt.imshow(diff[:, :, d], cmap='bwr', vmin=-0.1, vmax=0.1)

NameError: name 'volume_r2' is not defined

In [ ]:
nib

In [ ]:
wandb.run.notes

In [ ]:
r2_cv.max()

In [ ]:
volume = nsd.reconstruct_volume(subject_name, r2_cv, betas_indices)

@interact(d=(0, 60))
def show_betas(d):
    plt.figure(figsize=(12, 12))
    plt.imshow(volume.T[:, :, d], cmap='jet', vmin=0, vmax=0.5)

In [ ]:
load_betas_params = dict(
    subject_name='subj01',
    voxel_selection_path='derivatives/voxel-selection.hdf5',
    voxel_selection_key='nc/value',
    threshold=5.,
    return_volume_indices=True
)
betas, betas_indices = nsd.load_betas(**load_betas_params)
len(betas), betas_indices.shape

In [ ]:
r2 = r2_score(Y, Y_pred, reduction=None)

In [ ]:
volume = nsd.reconstruct_volume('subj01', r2, betas_indices)

@interact(d=(0, 60))
def show_betas(d):
    plt.figure(figsize=(12, 12))
    plt.imshow(volume.T[:, :, d], cmap='jet', vmin=0, vmax=0.5)

In [ ]:
wandb.run.history()

In [ ]:
experiment.val_dataset.indices

In [ ]:
subject_name = 'subj01'

In [ ]:
load_betas_params = dict(
    subject_name=subject_name,
    voxel_selection_path='derivatives/voxel-selection.hdf5',
    voxel_selection_key='nc/value',
    threshold=5.,
    return_volume_indices=True
)
betas, betas_indices = nsd.load_betas(**load_betas_params)
len(betas), betas_indices.shape

In [ ]:
@interact(i=(0, len(betas)-1), d=(0, 60))
def show_betas(i, d):
    volume = nsd.reconstruct_volume(subject_name, betas[i][0], betas_indices)
    plt.figure(figsize=(12, 12))
    plt.imshow(volume.T[:, :, d], cmap='bwr', vmin=-2, vmax=2)

In [ ]:
model_name = 'ViT-B=32'
load_stimulus_params = dict(
    subject_name=subject_name,
    #stimulus_path='nsddata_stimuli/stimuli/nsd/nsd_stimuli.hdf5',
    #stimulus_key='imgBrick',
    stimulus_path=f'derivatives/stimulus_embeddings/{model_name}.hdf5',
    stimulus_key='embedding',
    delay_loading=True
)
stimulus = nsd.load_stimulus(**load_stimulus_params)


In [ ]:
from torch.utils.data import random_split
dataset = KeyDataset({'betas': betas, 'stimulus': stimulus})
dataset, _, _ = nsd.apply_subject_split(dataset, subject_name, 'split-01')
train_dataset, val_dataset = nsd.apply_nfold_split(dataset, num_folds=5, select_fold=4)
len(train_dataset), len(val_dataset)

In [ ]:
from pipeline.utils import get_data_iterator

dataloader = DataLoader(train_dataset, shuffle=True, batch_size=12)
data_iterator = get_data_iterator(dataloader)

for i in range(20):
    x = next(data_iterator)
    print(x['betas'][0].shape, x['stimulus'][0].shape)

In [ ]:
spatial_shape = dataset.subjects['subj01']['betas']['betas'].attrs['spatial_shape']
betas.shape
betas = betas.reshape(10, *spatial_shape)

In [ ]:
@interact(c=(0, 10), d=(0, spatial_shape[-1]-1))
def show(c, d):
    volume = betas[c].T
    plt.imshow(volume[:, :, d])

In [ ]:
nc = h5py.File(dataset_path / 'derivatives/voxel-selection.hdf5', 'r')['subj01/nc/value'][:]


In [ ]:
@interact(d=(0, spatial_shape[-1]-1))
def show( d):
    plt.imshow(nc[:, :, d])

nc.shape

In [ ]:
import math

H = W = 14
C = 32
N = 4
V = 20

x = torch.randn(size=(N, C, H, W))

linear1 = nn.Linear(H * W, V)
w1 = torch.randn(size=(H, W, V)) / math.sqrt(H * W)
b1 = torch.zeros(size=(C, V))

linear2 = nn.Linear(C, V)
w2 = torch.randn(size=(C, V)) / math.sqrt(C)
b2 = torch.zeros(size=(V,))

#x1 = torch.einsum('nchw, hwv -> ncv', x, w1) + b1
#x2 = torch.einsum('ncv, cv -> nv', x1, w2) + b2

with torch.no_grad():
    x1 = torch.einsum('ncd, vd -> ncv', x.flatten(start_dim=2), linear1.weight) + linear1.bias
    x2 = torch.einsum('ncv, vc -> nv', x1, linear2.weight) + linear2.bias

print(f'{x.mean()=}, {x.std()=}')
print(f'{x1.mean()=}, {x1.std()=}')
print(f'{x2.mean()=}, {x2.std()=}')


In [ ]:
print(x.shape, linear1.weight.shape)

In [ ]:
linear2.weight.shape